In [ ]:
from langchain.schema import Document

# Wrap each document in the Document class
documents = [
    Document(page_content="Climate change affects polar bear habitats by reducing ice cover, making it harder for them to hunt seals."),
    Document(page_content="Rising global temperatures have led to the melting of polar ice caps, directly impacting wildlife in these regions."),
    Document(page_content="Various conservation efforts aim to protect polar bear populations by establishing protected areas and reducing greenhouse gas emissions."),
    Document(page_content="Climate change has far-reaching effects on ecosystems, impacting food chains and biodiversity on a global scale."),
    Document(page_content="Polar bears rely on sea ice for hunting, resting, and breeding. Loss of sea ice due to global warming forces them to travel farther for food.")
]


In [2]:
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

def standard_rag(documents, query, embedding_model_name="sentence-transformers/all-MiniLM-L6-v2", llm_model_name="EleutherAI/gpt-neo-125M", max_new_tokens=100, chain_type="stuff"):
    """
    Implements a standard RAG pipeline.
    
    Args:
        documents (list): List of `Document` objects containing page content.
        query (str): The query to retrieve information for.
        embedding_model_name (str): The name of the embedding model for vector storage. Default is "sentence-transformers/all-MiniLM-L6-v2".
        llm_model_name (str): The name of the language model for text generation. Default is "EleutherAI/gpt-neo-125M".
        max_new_tokens (int): The maximum number of tokens for generation. Default is 100.
        chain_type (str): The chain type to use for the RetrievalQA. Default is "stuff".
        
    Returns:
        str: The response generated for the query.
    """
    # Set up HuggingFaceEmbeddings with the specified model
    hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

    # Create the FAISS vector store using Document-wrapped documents
    vector_store = FAISS.from_documents(documents, hf_embeddings)

    # Define the retriever from the vector store
    retriever = vector_store.as_retriever()

    # Set up the generation pipeline
    generation_pipeline = pipeline("text-generation", model=llm_model_name, max_new_tokens=max_new_tokens)

    # Wrap the pipeline in LangChain as an LLM
    llm = HuggingFacePipeline(pipeline=generation_pipeline)

    # Create the RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type=chain_type
    )

    # Run the query through the chain and return the response
    response = qa_chain.run(query)
    return response


In [3]:
# Define the query
query = "What is the impact of climate change on polar bears?"

# Call the function
response = standard_rag(
    documents=documents,
    query=query
)

print(response)

c:\Users\Spandana\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Spandana\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Polar bears are the most vulnerable species in the world.

Climate change affects polar bear habitats by reducing ice cover, making it harder for them to hunt seals.

Polar bears rely on sea ice for hunting, resting, and breeding. Loss of sea ice due to global warming forces them to travel farther for food.

Various conservation efforts aim to protect polar bear populations by establishing protected areas and reducing greenhouse gas emissions.

Rising global temperatures have led to the melting of polar


In [7]:
def corrective_rag(documents: List[Document], query: str, embedding_model_name: str, llm_model_name: str):
    """
    Implements a Corrective Retrieval-Augmented Generation pipeline.
    """
    # Step 1: Initialize embeddings and vector store
    hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vector_store = FAISS.from_documents(documents, hf_embeddings)
    retriever = vector_store.as_retriever()

    # Step 2: Initialize the LLM for generation and semantic similarity
    generation_pipeline = pipeline("text-generation", model=llm_model_name, max_new_tokens=100)
    llm = HuggingFacePipeline(pipeline=generation_pipeline)

    def grade_relevance(query: str, retrieved_docs: List[Document]) -> bool:
        """
        Uses semantic similarity to grade the relevance of retrieved documents.
        """
        # Concatenate retrieved document content
        context = " ".join([doc.page_content for doc in retrieved_docs])
        grading_prompt = (
            f"Determine if the following context is relevant to the query:\n\n"
            f"Query: {query}\nContext: {context}\n\n"
            f"Respond with 'yes' or 'no'."
        )
        # Generate relevance grading using the LLM
        relevance_result = llm.generate([grading_prompt])
        relevance_response = relevance_result.generations[0][0].text  # Extract the first generation's text
        return "yes" in relevance_response.lower()

    def reflect_and_improve(response: str, query: str, retrieved_docs: List[Document]) -> str:
        """
        Implements a reflection mechanism to refine the output.
        """
        reflection_prompt = (
            f"Given the user's query: {query}\n\n"
            f"The initial response: {response}\n\n"
            f"Using the context: {' '.join([doc.page_content for doc in retrieved_docs])}, "
            f"improve the response to ensure it fully addresses the query."
        )
        # Generate refined response using the LLM
        reflection_result = llm.generate([reflection_prompt])
        refined_response = reflection_result.generations[0][0].text  # Extract the first generation's text
        return refined_response

    # Step 3: Retrieve initial documents
    retrieved_docs = retriever.get_relevant_documents(query)

    # Step 4: Check document relevance
    if not grade_relevance(query, retrieved_docs):
        # Rewrite the query for improved relevance
        refined_query = f"Provide more detailed context about: {query}"
        retrieved_docs = retriever.get_relevant_documents(refined_query)

    # Step 5: Generate initial response
    retrieval_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
    initial_response = retrieval_chain.run(query)

    # Step 6: Reflect and improve the response
    final_response = reflect_and_improve(initial_response, query, retrieved_docs)

    return final_response


In [8]:
query = "What is the impact of climate change on polar bears?"

response = corrective_rag(
    documents=documents,
    query=query,
    embedding_model_name="sentence-transformers/all-MiniLM-L6-v2",
    llm_model_name="EleutherAI/gpt-neo-125M"
)

print(response)


c:\Users\Spandana\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Polar bears rely on sea ice for hunting, resting, and breeding. Loss of sea ice due to global warming forces them to travel farther for food.

Polar bears rely on sea ice for hunting, resting, and breeding. Loss of sea ice due to global warming forces them to travel farther for food.

Polar bears rely on sea ice for hunting, resting, and breeding. Loss of sea ice due to global warming forces them to travel farther for food.



Embedding Model: sentence-transformers/all-MiniLM-L6-v2

Vector Store: FAISS

retriever: FAISS

LLM: gpt-neo-125M

framework: Langchain


types of RAG:  Standard RAG, Corrective RAG, Fusion RAG, Self RAG, Agentic RAG.